# ETH features selection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ===============================
# 1. LOAD DỮ LIỆU
# ===============================
print("Đang tải dữ liệu ETH...")
df = pd.read_csv('ETH_dataset.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)

# ===============================
# 2. TÍNH TOÁN CHỈ BÁO KỸ THUẬT (ĐÃ SỬA RSI CHUẨN WILDER)
# ===============================
def calculate_technical_indicators(df):
    df = df.copy()

    # Moving Averages
    df['SMA_20'] = df['close'].rolling(20).mean()
    df['SMA_50'] = df['close'].rolling(50).mean()
    df['EMA_12'] = df['close'].ewm(span=12, adjust=False).mean()
    df['EMA_26'] = df['close'].ewm(span=26, adjust=False).mean()

    # MACD
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # RSI chuẩn Wilder
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df['RSI_14'] = 100 - (100 / (1 + rs))

    # Bollinger Bands
    df['BB_middle'] = df['close'].rolling(20).mean()
    bb_std = df['close'].rolling(20).std()
    df['BB_upper'] = df['BB_middle'] + 2 * bb_std
    df['BB_lower'] = df['BB_middle'] - 2 * bb_std
    df['BB_position'] = (df['close'] - df['BB_lower']) / (df['BB_upper'] - df['BB_lower'])

    # Stochastic Oscillator
    low_14 = df['low'].rolling(14).min()
    high_14 = df['high'].rolling(14).max()
    df['STOCH_%K'] = 100 * (df['close'] - low_14) / (high_14 - low_14)
    df['STOCH_%D'] = df['STOCH_%K'].rolling(3).mean()

    # Williams %R
    df['Williams_%R'] = -100 * (high_14 - df['close']) / (high_14 - low_14)

    # ATR
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift())
    low_close = np.abs(df['low'] - df['close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = tr.rolling(14).mean()

    # Volume indicators
    df['Volume_SMA_20'] = df['volume'].rolling(20).mean()
    df['Volume_ratio'] = df['volume'] / df['Volume_SMA_20']

    # Rate of Change
    df['ROC_10'] = df['close'].pct_change(periods=10) * 100

    return df

df = calculate_technical_indicators(df)

# ===============================
# 3. CHUẨN BỊ FEATURES & TARGET
# ===============================
feature_columns = [
    'open', 'high', 'low', 'close', 'volume',
    'SMA_20', 'SMA_50', 'MACD', 'MACD_signal', 'RSI_14',
    'BB_position', 'STOCH_%K', 'Williams_%R', 'ATR_14',
    'Volume_ratio', 'ROC_10'
]

# Tạo target mượt (SMA_20) và nhãn xu hướng TRƯỚC khi dropna → tránh leakage
data = df[feature_columns].copy()
data['close_smooth'] = data['SMA_20']

# Loại bỏ các dòng NaN
data_clean = data.dropna().copy()


def feature_selection_vif(df, target_col='target', vif_threshold=10):
    
    # 1. Chuẩn bị dữ liệu
    # Lấy ra các biến số (numeric)
    df_numeric = df.select_dtypes(include=[np.number])
    
    # Xử lý NaN và Inf (VIF sẽ lỗi nếu có giá trị này)
    df_numeric = df_numeric.replace([np.inf, -np.inf], np.nan).dropna()
    
    if target_col not in df_numeric.columns:
        print(f"Lỗi: Không tìm thấy cột '{target_col}'")
        return None, None
        
    # Tách Features (X) và Target (y)
    # Lưu ý: Target KHÔNG tham gia vào quá trình tính VIF
    X = df_numeric.drop(columns=[target_col])
    
    print(f"--- Bắt đầu xử lý VIF với {X.shape[1]} đặc trưng ---")
    
    # 2. Vòng lặp loại bỏ biến có VIF cao nhất
    dropped_features = []
    
    while True:
        # Tính VIF cho tất cả các cột hiện tại
        vif_data = pd.DataFrame()
        vif_data["feature"] = X.columns
        
        # Công thức tính VIF
        try:
            vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                               for i in range(len(X.columns))]
        except Exception as e:
            print(f"Lỗi tính toán VIF (có thể do dữ liệu Constant/Null): {e}")
            break
            
        # Tìm feature có VIF lớn nhất
        max_vif = vif_data["VIF"].max()
        max_feature = vif_data.loc[vif_data["VIF"] == max_vif, "feature"].values[0]

        print(f"Max vif hiện tại: {max_vif}")
        
        # Kiểm tra điều kiện dừng
        if max_vif > vif_threshold:
            print(f"-> Loại bỏ '{max_feature}' (VIF = {max_vif:.2f})")
            X = X.drop(columns=[max_feature])
            dropped_features.append(max_feature)
        else:
            print(f"\n✅ Hoàn tất! Tất cả các biến còn lại đều có VIF < {vif_threshold}")
            break
    
    # 3. Đánh giá lại với Target (Bước cuối cùng)
    # Sau khi khử đa cộng tuyến, ta cần biết biến nào thực sự tác động đến Target
    final_features = X.columns.tolist()
    
    # Ghép lại với target để tính tương quan
    df_final = df_numeric[final_features + [target_col]]
    
    # Tính tương quan với target
    corr_to_target = df_final.corr()[target_col].drop(target_col).sort_values(ascending=False)
    
    print("\n--- Bảng VIF cuối cùng ---")
    print(vif_data.sort_values('VIF', ascending=False))
    
    print("\n--- Độ tương quan của các biến sạch với Target ---")
    print(corr_to_target)
    
    # Vẽ biểu đồ tương quan cuối cùng
    plt.figure(figsize=(10, 6))
    sns.heatmap(df_final.corr(), annot=True, cmap='RdBu_r', center=0)
    plt.title('Correlation Matrix (Sau khi lọc VIF)')
    plt.show()
    
    return df_final, vif_data


df_selected, vif_result = feature_selection_vif(data_clean, target_col='close_smooth', vif_threshold=10)



# onchain features selection

In [ ]:
# ==================== 1. KIỂM TRA & GÁN TARGET AN TOÀN ====================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestRegressor

eth_price = pd.read_csv("ETH_dataset.csv", parse_dates=['timestamp'])
onchain    = pd.read_csv("eth_onchain_features.csv", parse_dates=['date'])

# Kiểm tra nghiêm ngặt
if len(eth_price) != len(onchain):
    raise ValueError("Số dòng không bằng nhau → Không thể gán trực tiếp!")

if not (eth_price['timestamp'].dt.date == onchain['date'].dt.date).all():
    raise ValueError("Ngày không khớp → Bắt buộc phải merge theo date!")

print("OK: Ngày khớp hoàn toàn → Gán target an toàn")

# Gán target = giá đóng cửa ngày mai
df = onchain.drop(columns=['date']).copy()
df['target'] = eth_price['close'].shift(-1).values
df = df.dropna().reset_index(drop=True)

print(f"Dataset sẵn sàng: {len(df):,} ngày | Target = ETH close(t+1)\n")


# ==================== 2. HÀM VIF (đã hoàn hảo) ====================
def feature_selection_vif(df, target_col='target', vif_threshold=10):
    df_num = df.select_dtypes(include=[np.number]).copy()
    if target_col not in df_num.columns:
        raise ValueError(f"Không tìm thấy cột {target_col}")
    
    df_num = df_num.replace([np.inf, -np.inf], np.nan)
    features = [c for c in df_num.columns if c != target_col]
    X_clean = df_num[features].dropna()
    y_clean = df_num.loc[X_clean.index, target_col]
    X = X_clean.copy()
    
    print(f"Bắt đầu VIF với {X.shape[1]} features")
    dropped = []
    
    while True:
        vif_data = pd.DataFrame()
        vif_data["feature"] = X.columns
        vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
        
        max_vif = vif_data["VIF"].max()
        if max_vif <= vif_threshold:
            print(f"Hoàn thành VIF! Max VIF = {max_vif:.2f}")
            break
            
        feature_to_drop = vif_data.loc[vif_data["VIF"].idxmax(), "feature"]
        print(f"   Loại: {feature_to_drop} (VIF = {max_vif:.1f})")
        X = X.drop(columns=[feature_to_drop])
        dropped.append(feature_to_drop)
    
    final_df = pd.concat([X, y_clean.loc[X.index]], axis=1)
    print(f"\nĐã loại {len(dropped)} feature → Còn lại {X.shape[1]} feature sạch đa cộng tuyến\n")
    return final_df, dropped, vif_data


# ==================== 3. CHẠY VIF ====================
df_vif_clean, dropped_features, final_vif_table = feature_selection_vif(df, vif_threshold=10)

# Các feature còn lại sau VIF
clean_features = [col for col in df_vif_clean.columns if col != 'target']
print("Features sau khi loại đa cộng tuyến:")
print(clean_features)


# ==================== 4. RANDOM FOREST FEATURE IMPORTANCE (CHỈ TRÊN FEATURES SẠCH) ====================
X_clean = df_vif_clean[clean_features]
y_clean = df_vif_clean['target']

# Dùng toàn bộ dữ liệu sạch để Random Forest cho importance ổn định nhất
rf = RandomForestRegressor(
    n_estimators=3000,       # càng nhiều cây càng chính xác importance
    max_depth=None,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1,
    warm_start=False
)
rf.fit(X_clean, y_clean)

# Lấy importance
importances = pd.Series(rf.feature_importances_, index=clean_features).sort_values(ascending=False)

# ==================== 5. KẾT QUẢ ĐẸP NHẤT ====================
print("\n" + "="*70)
print("       TOP 15 ON-CHAIN FEATURES QUAN TRỌNG NHẤT ĐẾN GIÁ ETH NGÀY MAI")
print("                  (Sau khi đã loại đa cộng tuyến bằng VIF)")
print("="*70)
print(importances.head(15).round(4))

# Vẽ biểu đồ siêu đẹp
plt.figure(figsize=(10, 8))
sns.barplot(x=importances.head(15), y=importances.head(15).index, palette="rocket")
plt.title("Top 15 On-chain Features ảnh hưởng mạnh nhất đến ETH Price(t+1)\n"
          + "VIF-cleaned + Random Forest Importance", 
          fontsize=14, pad=20)
plt.xlabel("Importance Score", fontsize=12)
plt.tight_layout()
plt.show()

# Bonus: In toàn bộ ranking nếu muốn
# print("\nToàn bộ ranking:")
# print(importances.round(4))